In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [80]:
train_df=pd.read_csv("/kaggle/input/train.csv")
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [81]:
test_df=pd.read_csv("/kaggle/input/test.csv")
test_ids=test_df["PassengerId"]

In [82]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [83]:
train_df[["Sex","Survived"]].groupby(["Sex"]).mean().sort_values(by='Survived',ascending=False)

,Survived
Sex,
female,0.742038
male,0.188908


In [84]:
train_df[["Pclass","Survived"]].groupby(["Pclass"]).mean().sort_values('Survived',ascending=False)

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [85]:
train_df["Age"].fillna(train_df["Age"].mean())
train_df[["Embarked","Survived"]].groupby(["Embarked"]).mean().sort_values('Survived',ascending=False)

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [86]:
train_df["Family"]=train_df["SibSp"]+train_df["Parch"]
train_df["Family"]

0       1
1       1
2       0
3       1
4       0
5       0
6       0
7       4
8       2
9       1
10      2
11      0
12      0
13      6
14      0
15      0
16      5
17      0
18      1
19      0
20      0
21      0
22      0
23      0
24      4
25      6
26      0
27      5
28      0
29      0
       ..
861     1
862     0
863    10
864     0
865     0
866     1
867     0
868     0
869     2
870     0
871     2
872     0
873     0
874     1
875     0
876     0
877     0
878     0
879     1
880     1
881     0
882     0
883     0
884     0
885     5
886     0
887     0
888     3
889     0
890     0
Name: Family, Length: 891, dtype: int64

In [87]:
train_df2=train_df.drop(["Cabin","SibSp","Parch","Ticket","PassengerId"], axis="columns")
train_df2["Age"]=train_df2["Age"].fillna(train_df2["Age"].median())
train_df2["Age"]=train_df2["Age"].round()
train_df2["Age"]
train_df2["Embarked"]=train_df2["Embarked"].fillna("S")

In [88]:
logreg = LogisticRegression()
Y_train=train_df2["Survived"]
X_train=train_df2.drop("Survived",axis="columns")


In [89]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
Fare        891 non-null float64
Embarked    891 non-null object
Family      891 non-null int64
dtypes: float64(2), int64(2), object(3)
memory usage: 48.8+ KB


In [90]:
X_train

,Pclass,Name,Sex,Age,Fare,Embarked,Family
0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,S,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C,1
2,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,S,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,S,1
4,3,"Allen, Mr. William Henry",male,35.0,8.0500,S,0
5,3,"Moran, Mr. James",male,28.0,8.4583,Q,0
6,1,"McCarthy, Mr. Timothy J",male,54.0,51.8625,S,0
7,3,"Palsson, Master. Gosta Leonard",male,2.0,21.0750,S,4
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,11.1333,S,2
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,30.0708,C,1


In [91]:
X_train=X_train.drop("Name", axis ="columns")

In [92]:
X_train

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,male,22.0,7.2500,S,1
1,1,female,38.0,71.2833,C,1
2,3,female,26.0,7.9250,S,0
3,1,female,35.0,53.1000,S,1
4,3,male,35.0,8.0500,S,0
5,3,male,28.0,8.4583,Q,0
6,1,male,54.0,51.8625,S,0
7,3,male,2.0,21.0750,S,4
8,3,female,27.0,11.1333,S,2
9,2,female,14.0,30.0708,C,1


In [93]:
dummies= pd.get_dummies(X_train.Sex)
X_train=X_train.drop("Sex",axis="columns")
X_train=pd.concat([X_train,dummies],axis="columns")
scaling=MinMaxScaler()
X_train[['Age','Fare']]=scaling.fit_transform(X_train[['Age','Fare']])
X_train


,Pclass,Age,Fare,Embarked,Family,female,male
0,3,0.2750,0.014151,S,1,0,1
1,1,0.4750,0.139136,C,1,1,0
2,3,0.3250,0.015469,S,0,1,0
3,1,0.4375,0.103644,S,1,1,0
4,3,0.4375,0.015713,S,0,0,1
5,3,0.3500,0.016510,Q,0,0,1
6,1,0.6750,0.101229,S,0,0,1
7,3,0.0250,0.041136,S,4,0,1
8,3,0.3375,0.021731,S,2,1,0
9,2,0.1750,0.058694,C,1,1,0


In [94]:
dummies2= pd.get_dummies(X_train.Embarked)
X_train=X_train.drop("Embarked",axis="columns")
X_train=pd.concat([X_train,dummies2],axis="columns")
X_train

,Pclass,Age,Fare,Family,female,male,C,Q,S
0,3,0.2750,0.014151,1,0,1,0,0,1
1,1,0.4750,0.139136,1,1,0,1,0,0
2,3,0.3250,0.015469,0,1,0,0,0,1
3,1,0.4375,0.103644,1,1,0,0,0,1
4,3,0.4375,0.015713,0,0,1,0,0,1
5,3,0.3500,0.016510,0,0,1,0,1,0
6,1,0.6750,0.101229,0,0,1,0,0,1
7,3,0.0250,0.041136,4,0,1,0,0,1
8,3,0.3375,0.021731,2,1,0,0,0,1
9,2,0.1750,0.058694,1,1,0,1,0,0


In [95]:
logreg = LogisticRegression()
X_train,X_test,y_train,y_test= train_test_split(X_train,Y_train,test_size=0.2,random_state=42)
logreg.fit(X_train,y_train)
logreg.predict(X_test)
logreg.score(X_test,y_test)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8100558659217877

In [96]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
decision_tree.predict(X_test)
logreg.score(X_test,y_test)


0.8100558659217877

In [97]:
svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test,y_test)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7988826815642458

In [98]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.8044692737430168

In [99]:
test_df["Family"]=test_df["SibSp"]+test_df["Parch"]
test_df["Family"]

0      0
1      1
2      0
3      0
4      2
5      0
6      0
7      2
8      0
9      2
10     0
11     0
12     1
13     1
14     1
15     1
16     0
17     0
18     1
19     0
20     1
21     1
22     0
23     1
24     4
25     1
26     1
27     0
28     0
29     2
      ..
388    0
389    4
390    0
391    1
392    2
393    0
394    4
395    1
396    0
397    2
398    0
399    0
400    0
401    1
402    1
403    0
404    1
405    0
406    1
407    2
408    0
409    2
410    0
411    1
412    0
413    0
414    0
415    0
416    0
417    2
Name: Family, Length: 418, dtype: int64

In [100]:
test_df["Age"]=test_df["Age"].fillna(train_df2["Age"].median())
test_df["Age"]=test_df["Age"].round()
test_df=test_df.drop(["Cabin","SibSp","Parch","Ticket","PassengerId"], axis="columns")
dummiest1= pd.get_dummies(test_df.Sex)
test_df=test_df.drop("Sex",axis="columns")
test_df=pd.concat([test_df,dummiest1],axis="columns")
dummiest2= pd.get_dummies(test_df.Embarked)
test_df=test_df.drop("Embarked",axis="columns")
test_df=pd.concat([test_df,dummiest2],axis="columns")
test_df=test_df.drop("Name",axis="columns")
test_df["Fare"]=test_df["Fare"].fillna(test_df["Fare"].median())
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
Pclass    418 non-null int64
Age       418 non-null float64
Fare      418 non-null float64
Family    418 non-null int64
female    418 non-null uint8
male      418 non-null uint8
C         418 non-null uint8
Q         418 non-null uint8
S         418 non-null uint8
dtypes: float64(2), int64(2), uint8(5)
memory usage: 15.2 KB


In [101]:
test_df[['Age','Fare']]=scaling.fit_transform(test_df[['Age','Fare']])    
test_df

,Pclass,Age,Fare,Family,female,male,C,Q,S
0,3,0.447368,0.015282,0,0,1,0,1,0
1,3,0.618421,0.013663,1,1,0,0,0,1
2,2,0.815789,0.018909,0,0,1,0,1,0
3,3,0.355263,0.016908,0,0,1,0,0,1
4,3,0.289474,0.023984,2,1,0,0,0,1
5,3,0.184211,0.018006,0,0,1,0,0,1
6,3,0.394737,0.014891,0,1,0,0,1,0
7,2,0.342105,0.056604,2,0,1,0,0,1
8,3,0.236842,0.014110,0,1,0,1,0,0
9,3,0.276316,0.047138,2,0,1,0,0,1


In [103]:
rf= RandomForestClassifier(n_estimators=90,max_leaf_nodes=60)
rf.fit(X_train,y_train)
print(rf.score(X_test,y_test))
submission_preds=rf.predict(test_df)
df=pd.DataFrame({"PassengerId":test_ids.values,"Survived":submission_preds})
df.to_csv("submission.csv",index=False) 

0.8324022346368715


In [104]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,86.76
8,Decision Tree,86.76
1,KNN,84.74
0,Support Vector Machines,83.84
2,Logistic Regression,80.36
6,Stochastic Gradient Decent,79.35
7,Linear SVC,79.01
5,Perceptron,78.00
4,Naive Bayes,72.28
